In [ ]:

# Bỏ qua tất cả các cảnh báo
warnings.filterwarnings('ignore')

# Đọc dữ liệu
features_df = features_train_df

# Loại bỏ cột 'label' khỏi tập đặc trưng
features = features_df.drop(columns=['label'])

# Chuyển đổi các giá trị đặc trưng thành dạng rời rạc
discretizer = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
discrete_features = discretizer.fit_transform(features)

# Chuyển đổi ngược lại thành DataFrame để dễ xử lý
discrete_features_df = pd.DataFrame(discrete_features, columns=features.columns)

# Tạo danh sách giao dịch có gắn nhãn đặc trưng
transactions_list = [[f'{col}_{int(val)}' for col, val in row.items()] for index, row in discrete_features_df.iterrows()]

# Sử dụng TransactionEncoder để chuyển đổi danh sách giao dịch thành DataFrame nhị phân
te = TransactionEncoder()
binary_features_df = pd.DataFrame(te.fit(transactions_list).transform(transactions_list), columns=te.columns_)

# Sử dụng thuật toán Apriori để tìm các tập hợp thường xuyên
frequent_itemsets = apriori(binary_features_df, min_support=0.6, use_colnames=True)

# Tạo các luật kết hợp từ các tập hợp thường xuyên
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)

# Đánh giá các luật dựa trên các chỉ số
min_support = 0.6
min_confidence = 0.7
min_lift = 1.0
evaluated_rules = rules[(rules['support'] >= min_support) & (rules['confidence'] >= min_confidence) & (rules['lift'] >= min_lift)]

# Xác định các thuộc tính quan trọng từ các luật kết hợp
important_features = pd.Series(evaluated_rules['antecedents'].tolist() + evaluated_rules['consequents'].tolist()).value_counts()
important_features_df = important_features.reset_index()
important_features_df.columns = ['attribute', 'count']

# Chuyển đổi frozenset thành chuỗi và lấy tên các thuộc tính gốc
def extract_feature_names(frozenset_str):
    return [feat.split('_')[0] for feat in frozenset_str.replace('frozenset({', '').replace('})', '').replace("'", "").replace('(', '').replace(')', '').split(', ')]

important_original_features = set()
for item in important_features_df['attribute'].tolist():
    if isinstance(item, str) and 'frozenset' in item:
        item = eval(item)
    if isinstance(item, frozenset):
        for sub_item in item:
            important_original_features.update(extract_feature_names(sub_item))
    else:
        important_original_features.update(extract_feature_names(item))

important_original_features = list(important_original_features)
important_original_features.append('label')

print("Các thuộc tính gốc quan trọng:")
print(important_original_features)



Các thuộc tính gốc quan trọng:
['correlation', 'entropy', 'energy', 'homogeneity', 'contrast', 'label']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Lấy các giá trị duy nhất trong cột attribute
unique_attributes = important_features_df['attribute'].unique()

# Chuyển đổi frozenset thành danh sách các thuộc tính gốc dễ hiểu
important_features_list = set()
for attr in unique_attributes:
    if isinstance(attr, frozenset):
        important_features_list.update(attr)
    else:
        important_features_list.add(attr)
# Hiển thị danh sách các thuộc tính gốc dễ hiểu
print("Danh sách các thuộc tính gốc dễ hiểu :")
print(important_features_list)

Danh sách các thuộc tính gốc dễ hiểu :
{'entropy_intensity_3', 'homogeneity_0_0', 'energy_45_0', 'contrast_135_0', 'correlation_90_0', 'energy_intensity_6'}


In [ ]:
important_features = ['entropy_intensity', 'energy_intensity','correlation_90', 'contrast_135','variance_intensity','homogeneity_0','energy_45','label']

In [ ]:
features_train_df = features_train_df[important_features]
features_test_df = features_test_df[important_features]

In [ ]:
# Tạo tập dữ liệu từ F/s gốc - PP2
X_train = features_train_df.drop('label', axis=1)
y_train = features_train_df['label']
X_test = features_test_df.drop('label', axis=1)
y_test = features_test_df['label']

In [ ]:
# Combine train and test datasets
combined_df = pd.concat([features_train_df, features_test_df])
combined_df = combined_df[important_features]
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1367 entries, 0 to 133
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   entropy_intensity   1367 non-null   float64
 1   energy_intensity    1367 non-null   float64
 2   correlation_90      1367 non-null   float64
 3   contrast_135        1367 non-null   float64
 4   variance_intensity  1367 non-null   float64
 5   homogeneity_0       1367 non-null   float64
 6   energy_45           1367 non-null   float64
 7   label               1367 non-null   object 
dtypes: float64(7), object(1)
memory usage: 96.1+ KB


In [ ]:
X_combined = combined_df.drop('label', axis=1)
y_combined = combined_df['label']

# Split the combined dataset into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_combined, test_size=0.2, random_state=42, stratify=y_combined)